In [15]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

## SHOW IMAGE

In [16]:
def display_image(image, figsize=(3, 3)):
    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

## DEFINE MODEL

In [17]:
box_number_model = YOLO("./models/box_number_model.pt")
number_model = YOLO("./models/number_2.pt")
dot_box_model = YOLO("./models/circle_number_model.pt")
dot_model = YOLO("./models/final_model.pt")
dot_boxes_model = YOLO("./models/dot_boxes_detection.pt")

## GENERATE IMAGE VARIATONS

Membuat beberapa variasi gambar dari satu gambar kotak angka

In [18]:
def generate_variations(image):
    variations = []
    
    # Variasi rotasi
    rows, cols = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), 11, 1)  # Rotasi 11 derajat
    rotated = cv2.warpAffine(image, rotation_matrix, (cols, rows))
    variations.append(rotated)

    # Variasi blur
    blurred = cv2.GaussianBlur(image, (5, 5), 2)  # Blur 2px
    variations.append(blurred)

    # Variasi grayscale
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    variations.append(cv2.cvtColor(grayscale, cv2.COLOR_GRAY2BGR))
    
    # Resize Varians
    resize = cv2.resize(image, (640*2,640*2))
    variations.append(resize)

    return variations

## ROTATE IMAGE

Melakukan rotasi gambar apabila terdeteksi terbalik berdasrkan nilai x pada kotak bagian suara yang harusnya di kanan. Apabila terdeteksi di kiri dengan patokan nilai < lebar / 2 maka balik gambar

In [19]:
def rotate_image(image):
    detection = dot_box_model(image)[0].boxes.data.tolist()[0] # Hanya deteksi indeks 0

    x1, y1, x2, y2, score, class_id = detection
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    if x1 < image.shape[1] / 2:  # Jika x1 berada di sebelah kiri gambar
        image = cv2.rotate(image, cv2.ROTATE_180)

    return image

## GET NUMBER FROM NUMBER IMAGE

### GET NUMBER BOXES

Mendapatkan kotak berisi angka suara untuk dideteksi angka di dalamnya

In [20]:
def get_number_by_box(image):
    number_box_detection = box_number_model(image)[0]
    sorted_number_box_detection = sorted(number_box_detection.boxes.data.tolist(), key=lambda x: x[1])
    
    number_boxes = []
    
    for number_box in sorted_number_box_detection:
        number_x1, number_y1, number_x2, number_y2, number_score, number_class_id = number_box
        number_x1, number_y1, number_x2, number_y2 = int(number_x1), int(number_y1), int(number_x2), int(number_y2)
        
        number_box_image = image[number_y1:number_y2, number_x1:number_x2]
        number_box_image = cv2.resize(number_box_image, (720,480))
        
        # display_image(number_box_image)
        
        number_boxes.append(number_box_image)
    
    return number_boxes

### DETECT NUMBER FROM NUMBER IMAGE

Mendeteksi dan menyusun suara yang terdeteksi oleh model

In [21]:
def get_number_from_image(images):
    class_key = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '0'}
    
    semua_suara = []
    for image in images:
        variations = generate_variations(image)
        varians_suara = []
        for var_image in variations:
            number_detection = number_model(var_image)[0]
            # display_image(var_image)
            suara = []
            for number in sorted(number_detection.boxes.data.tolist(), key=lambda x: x[0]):
                x1, y1, x2, y2, score, class_id = number
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                suara.append(class_key[class_id])
            
            if len(number_detection) != 0:
                joined_suara = ''.join(suara)
                suara_as_int = int(joined_suara)  
                varians_suara.append(suara_as_int)
            else:
                varians_suara.append(0)
    
        semua_suara.append(varians_suara)
              
    return semua_suara

## GET NUMBER FROM DOTS IMAGE

### SPLIT DOTES BOX

Melakukan crop pada kotak berisi titik titik hitam menjadi seukuran dengan jumlah kotak suara

In [22]:
def split_dot_boxes(image, split_image):
    width = image.shape[1]  # Mendapatkan lebar gambar

    part_width = width // split_image
    grouped_images = []

    for i in range(split_image):
        start = i * part_width
        end = (i + 1) * part_width if i < split_image - 1 else width
        dot_boxes_cropped = image[:, start:end]  # Mengambil bagian gambar
        # display_image(dot_boxes_cropped)
        grouped_images.append(dot_boxes_cropped)

    return grouped_images

### DETECT NUMBER FROM DOTES

Mendeteksi suara berdasarkan posisi titik hitam berada

In [23]:
def detect_number(images):
    skor_dot = []

    for dot_box_crop in images:
        detected = False
        part_height = dot_box_crop.shape[0] // 10
        
        for i, _ in enumerate(range(10)):
            part = dot_box_crop[i * part_height: (i + 1) * part_height, :]
            # display_image(part)
                  
            gray_image = cv2.cvtColor(part, cv2.COLOR_BGR2GRAY)
                
            # Terapkan thresholding
            _, binary_image = cv2.threshold(gray_image, 112, 255, cv2.THRESH_BINARY)
            
            # display_image(binary_image)
            
            # Hitung jumlah piksel hitam
            black_pixel_count = np.sum(binary_image == 0)
            
            # Hitung jumlah total piksel
            total_pixels = binary_image.size
            
            # Hitung persentase piksel hitam
            black_pixel_percentage = (black_pixel_count / total_pixels) * 100
            
            # Cek apakah titik terdeteksi berdasarkan persentase piksel hitam
            if 10 <= black_pixel_percentage < 80:  # Menyesuaikan ambang batas
                detected = True  
                skor_dot.append(str(i))
                break
        
    if detected == False:
        skor_dot.append("0")
        
    return ''.join(skor_dot)

### GET DOTES BOX

Mendeteksi kotak titik kemudian mencari tau apakah merupakan kertas suara maksimal ratusan suara atau ribuan suara. Jika ratusan maka berikan instruksi bahwa kotak titik dipotong jadi tiga, jika tidak maka potong menjadi 5

In [24]:
def get_dots_boxes(image):
    dot_boxes = dot_boxes_model(image)[0].boxes.data.tolist()

    if len(dot_boxes) <= 9:
        split_image = 3
    else:
        split_image = 5
    
    dot_boxes_detection = dot_box_model(image)[0]
    
    sorted_dot_box_detection = sorted(dot_boxes_detection.boxes.data.tolist(), key=lambda x: x[1])

    splited_dot_image = []

    for dot_box in sorted_dot_box_detection:
        # display_image
        dot_x1, dot_y1, dot_x2, dot_y2, dot_score, dot_class_id = dot_box
        dot_x1, dot_y1, dot_x2, dot_y2 = int(dot_x1), int(dot_y1), int(dot_x2), int(dot_y2)
        
        dot_box_image = image[dot_y1:dot_y2, dot_x1:dot_x2]   
        
        # display_image(dot_box_image)
        
        splitted_box = split_dot_boxes(dot_box_image, split_image)
    
        splited_dot_image.append(splitted_box)
    
    dot_suara_paslon = []
    for split_image in splited_dot_image:
        number = detect_number(split_image)
        dot_suara_paslon.append([int(number),int(number),int(number),int(number)])
        
    return dot_suara_paslon

## MAIN PROGRAM

In [ ]:

path = "./dataset/Test"
data = os.listdir(path)

persentase = []
Nama_TPS = []
semua_paslon = []
paslon_satu = []
paslon_dua = []
paslont_tiga = []

for TPS in data:
        # LOAD IMAGE
    image = cv2.imread(f"./dataset/Test/{TPS}")

    # display_image(image)

    # CHECK IF NEEDED ROTATE
    image = rotate_image(image)

    # # CROPED NUMBER BOXES
    number_boxes = get_number_by_box(image)

    # # GET NUMBER FROM IMAGE
    number_from_image = get_number_from_image(number_boxes)
    
    print(number_from_image)
    
    # GET NUMBER FROM DOT
    number_from_dot = [get_dots_boxes(image)][0]

    # COMBINED VOTE (Menggabungkan prediksi by image dan titik)
    combined_vote = []
    for sublist1, sublist2 in zip(number_from_image, []):
        combined_sublist = sublist1 + sublist2
        combined_vote.append(combined_sublist)
        
    print(combined_vote)
    
    # FILTER INVALID VOTE seperti ribuan     
    vote_result = []
    for sublist in number_from_image:
        sublist_without_zeros = list(filter(lambda x: x != 0 and x < 1000, sublist))
        vote_result.append(sublist_without_zeros)

    # MEMILIH UNTUK MENGAMBIL MODUS DARI SETIAP PEMILIH
    final_result = []
    for i in vote_result:
        if len(i) == 0:
            final_result.append(0)
        else:
            final_result.append(max(set(i), key=i.count))

    semua_paslon.append({TPS : final_result})
    paslon_satu.append(final_result[0])
    paslon_dua.append(final_result[1])
    paslont_tiga.append(final_result[2])
   paslon_dua_persentase = final_resut[1]l
    total_suara = sum(final_result)
    persentase.append(round((paslon_dua_persentase / total_suara) * 100,4)) 
    Nama_TPS.append(TPS.split('.')[0]) 

In [57]:
semua_paslon

[{'TPS_501.jpg': [77, 141, 18]},
 {'TPS_502.jpg': [122, 98, 10]},
 {'TPS_503.jpg': [41, 165, 23]},
 {'TPS_504.jpg': [21, 192, 41]},
 {'TPS_505.jpg': [69, 112, 11]},
 {'TPS_506.jpg': [68, 120, 31]},
 {'TPS_507.jpg': [40, 170, 47]},
 {'TPS_508.jpg': [47, 126, 38]},
 {'TPS_509.jpg': [48, 128, 32]},
 {'TPS_510.jpg': [40, 167, 10]},
 {'TPS_511.jpg': [10, 163, 48]},
 {'TPS_512.jpg': [54, 172, 27]},
 {'TPS_513.jpg': [29, 202, 8]},
 {'TPS_514.jpg': [29, 161, 58]},
 {'TPS_515.jpg': [78, 31, 39]},
 {'TPS_516.jpg': [73, 474, 84]},
 {'TPS_517.jpg': [22, 183, 42]},
 {'TPS_518.jpg': [46, 154, 32]},
 {'TPS_519.jpg': [34, 109, 49]},
 {'TPS_520.jpg': [19, 170, 53]},
 {'TPS_521.jpg': [14, 143, 49]},
 {'TPS_522.jpg': [47, 149, 34]},
 {'TPS_523.jpg': [58, 137, 41]},
 {'TPS_524.jpg': [26, 157, 58]},
 {'TPS_525.jpg': [8, 166, 30]},
 {'TPS_526.jpg': [180, 176, 88]},
 {'TPS_527.jpg': [11, 162, 71]},
 {'TPS_528.jpg': [8, 104, 42]},
 {'TPS_529.jpg': [10, 141, 43]},
 {'TPS_530.jpg': [9, 148, 41]},
 {'TPS_531.jpg

In [27]:
# data = {
#     "TPS": Nama_TPS,
#     "Persentase Suara Sah Paslon 2": persentase
# }

# # Membuat DataFrame
# jawaban = pd.DataFrame(data)

# jawaban.to_csv("gammafest_results2.csv",index=False)

In [28]:
jawaban

NameError: name 'jawaban' is not defined

In [ ]:
# Membaca file CSV dengan jawaban asli dan hasilnya
df_jawaban = pd.read_csv("kunci_jawaban.csv")
df_hasil = jawaban

# Menghitung selisih absolut antara setiap baris
df_selisih = abs(df_jawaban.iloc[:, 1:] - df_hasil.iloc[:, 1:])

# Menjumlahkan selisih absolut dari setiap baris
df_total_selisih = df_selisih.sum(axis=1)

# Menjadikan selisih negatif menjadi positif
df_total_selisih = df_total_selisih.apply(lambda x: abs(x))

# Menghitung total jawaban yang benar
total_jawaban_benar = df_jawaban.iloc[:, 1:].sum().sum()

# Menghitung total error
total_error = df_total_selisih.sum()

# Menghitung persentase error dari jawaban benar
persentase_error = (total_error / total_jawaban_benar) * 100
print("Persentase error dari jawaban benar:", persentase_error)

Persentase error dari jawaban benar: 0.0


In [ ]:
suara = {
    "tps": Nama_TPS,
    "suara_paslon_1": paslon_satu,
    "suara_paslon_2" : paslon_dua,
    "suara_paslon_3" : paslont_tiga
}

# Membuat DataFrame
suara_paslon = pd.DataFrame(suara)

suara_paslon.to_csv("Prediksi_Nama Tim.csv",index=False)    